# CIFAR-10 Image Classifier

## 1. Import Libraries

In [1]:
import torch
from torch import nn

import torchvision
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import OneCycleLR

import matplotlib.pyplot as plt


In [2]:
# setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"

### Data Transforming

In [3]:
# Data Transforming as for CIFAR10 datasets
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

## 2. Load Dataset

In [4]:
# Load CIFAR-10 dataset
trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=2)

100%|██████████| 170M/170M [00:03<00:00, 43.0MB/s]


## 3. Model Architecture

In [5]:
class CustomModel(nn.Module):
    def __init__(self, num_classes=10):
        super(CustomModel, self).__init__()

        # Stage 1: 32 filters
        self.stage1 = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        # Stage 2: 64 filters
        self.stage2 = nn.Sequential(
            nn.Conv2d(32, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        # Stage 3: 128 filters
        self.stage3 = nn.Sequential(
            nn.Conv2d(64, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        # Global Average Pooling + Classifier
        self.gap = nn.AdaptiveAvgPool2d(1)
        self.classifier = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.stage1(x)
        x = self.stage2(x)
        x = self.stage3(x)
        x = self.gap(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x



In [6]:
# creating Model
model = CustomModel().to(device)

In [7]:
# train setup( loss function & )
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(),lr = 0.0003 ,weight_decay= 0.0001 )

# Learning rate sceduler
scheduler = OneCycleLR(optimizer, max_lr= 0.001,
                       epochs= 80,
                       steps_per_epoch= len(trainloader))

# accuracy function
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item()
    acc = (correct / len(y_pred)) * 100
    return acc

## 4. Training

In [8]:
# Training function
def train(epoch):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        scheduler.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

    acc = 100. * correct / total
    if epoch % 10 == 0:
      print(f'Epoch: {epoch} | Loss: {running_loss/len(trainloader):.3f} | Acc: {acc:.3f}%')


In [9]:
# Test function
def test(epoch):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, targets in testloader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    acc = 100. * correct / total
    if epoch % 10 == 0:
      print(f'Test Accuracy: {acc:.3f}%')
    return acc

In [10]:
print("Starting training...")
best_acc = 0
for epoch in range(1, 31):
    train(epoch)
    acc = test(epoch)

    if acc > best_acc:
        best_acc = acc
        print(f'New best accuracy: {best_acc:.3f}% on {epoch} point')

print(f'Best test accuracy: {best_acc:.3f}%')

Starting training...
New best accuracy: 45.590% on 1 point
New best accuracy: 54.370% on 2 point
New best accuracy: 57.910% on 3 point
New best accuracy: 59.500% on 4 point
New best accuracy: 62.550% on 5 point
New best accuracy: 66.610% on 7 point
New best accuracy: 68.950% on 8 point
Epoch: 10 | Loss: 0.732 | Acc: 74.512%
Test Accuracy: 66.410%
New best accuracy: 71.110% on 11 point
New best accuracy: 72.540% on 12 point
New best accuracy: 76.480% on 13 point
New best accuracy: 77.310% on 15 point
New best accuracy: 78.860% on 18 point
Epoch: 20 | Loss: 0.460 | Acc: 84.150%
Test Accuracy: 76.840%
New best accuracy: 79.650% on 21 point
New best accuracy: 82.060% on 22 point
New best accuracy: 82.780% on 25 point
New best accuracy: 84.400% on 26 point
New best accuracy: 85.010% on 29 point
Epoch: 30 | Loss: 0.325 | Acc: 88.808%
Test Accuracy: 83.670%
Best test accuracy: 85.010%


## 5. Evaluation & Results

In [11]:
def evaluate_model(model, testloader, device):
    """
    Evaluate the model on test dataset
    """
    model.eval()
    correct = 0
    total = 0
    all_predictions = []
    all_targets = []

    with torch.no_grad():
        for inputs, targets in testloader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            # Store predictions and targets for detailed analysis
            all_predictions.extend(predicted.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())

    accuracy = 100. * correct / total
    return accuracy, all_predictions, all_targets

# Usage after training:
print("Evaluating model...")
test_accuracy, predictions, targets = evaluate_model(model, testloader, device)
print(f'Final Test Accuracy: {test_accuracy:.3f}%')

# Additional evaluation: Class-wise accuracy
def class_wise_accuracy(model, testloader, device, classes):
    """
    Calculate accuracy for each class
    """
    model.eval()
    class_correct = list(0. for i in range(10))
    class_total = list(0. for i in range(10))

    with torch.no_grad():
        for inputs, targets in testloader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            c = (predicted == targets).squeeze()

            for i in range(targets.size(0)):
                label = targets[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1

    print("\nClass-wise Accuracy:")
    for i in range(10):
        if class_total[i] > 0:
            print(f'{classes[i]:12s}: {100 * class_correct[i] / class_total[i]:.2f}%')

# CIFAR-10 classes
classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

# Get class-wise accuracy
class_wise_accuracy(model, testloader, device, classes)

Evaluating model...
Final Test Accuracy: 83.670%

Class-wise Accuracy:
plane       : 72.70%
car         : 88.50%
bird        : 76.60%
cat         : 78.90%
deer        : 92.10%
dog         : 71.20%
frog        : 83.00%
horse       : 89.70%
ship        : 93.20%
truck       : 90.80%
